# Colab with Flask

## Train MLP

In [7]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

### preprocess data

In [9]:
# load data
X=pd.read_csv('X.csv')

with open ('y.npy', 'rb') as f:
  y=np.load(f)

In [11]:
# select features
# 전부 다 쓰지 않고 8개만 선택
X=X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

In [12]:
X.shape

(1460, 8)

In [18]:
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y=y_min_max_scaler.transform(y)

In [19]:
scaled_X.shape, scaled_y.shape

((1460, 8), (1460, 1))

### training

In [22]:
model = keras.Sequential(
    [
        keras.Input(shape=scaled_X.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
    ]
)

model.compile(loss="mse", optimizer="adam")

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.fit(scaled_X, scaled_y,
          batch_size=2, epochs=150,
          callbacks=[early_stopping_callback], validation_split=0.005)

Epoch 1/150
726/726 [==============================] - 3s 3ms/step - loss: 0.0043 - val_loss: 0.0016
Epoch 2/150
726/726 [==============================] - 3s 4ms/step - loss: 0.0031 - val_loss: 0.0018
Epoch 3/150
726/726 [==============================] - 2s 2ms/step - loss: 0.0033 - val_loss: 0.0032
Epoch 4/150
726/726 [==============================] - 2s 2ms/step - loss: 0.0030 - val_loss: 0.0014
Epoch 5/150
726/726 [==============================] - 2s 2ms/step - loss: 0.0027 - val_loss: 0.0011
Epoch 6/150
726/726 [==============================] - 2s 2ms/step - loss: 0.0026 - val_loss: 0.0013
Epoch 7/150
726/726 [==============================] - 2s 2ms/step - loss: 0.0026 - val_loss: 0.0021
Epoch 8/150
726/726 [==============================] - 2s 2ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 9/150
726/726 [==============================] - 3s 4ms/step - loss: 0.0025 - val_loss: 0.0010
Epoch 10/150
726/726 [==============================] - 2s 2ms/step - loss: 0.0022 - val_lo

In [23]:
scaled_X.shape

(1460, 8)

In [24]:
pred=model.predict(scaled_X[:5])
pred=y_min_max_scaler.inverse_transform(pred)

1/1 [==============================] - 0s 121ms/step


In [25]:
pred.shape

(5, 1)

In [26]:
str(pred[0][0])

'185372.31'

### Save MLP model

In [30]:
# model 저장해서 쓰면 더 효율적
# 학습 시킨 모델은 일반적으로 h5 파일로 저장함
# H5 파일은 HDF (Hierarchical Data Format)로 저장된 데이터 파일
model.save("mlp_v0.1.h5")

In [32]:
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

In [33]:
pred = reconstructed_model.predict(scaled_X[:1]) #0-1
pred = y_min_max_scaler.inverse_transform(pred)

1/1 [==============================] - 0s 72ms/step


In [34]:
pred

array([[185372.31]], dtype=float32)

## install Flask Ngrok

In [27]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Flask Web Server 만들기

- 아주 간단한 형태임

In [28]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app=Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
  return "<h1>This is your Flask server.</h1>"

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0291-34-80-93-56.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [29]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# 서버가 실행 될 때 X, y 데이터 가지고 있어야 좋다
# load data
X=pd.read_csv('X.csv')

with open ('y.npy', 'rb') as f:
  y=np.load(f)

X=X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

# Min Max Scaler 하면 서버가 시작할 때 최대 최소값을 가지고 있게 됨
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)

# load model
reconstructed_model=keras.models.load_model("mlp_v0.1.h5")


# run Server
app=Flask(__name__, template_folder='/content')
run_with_ngrok(app)

def preprocess_data(data):
  #TODO : preprocessing

  # Dictionary --> np array (1, 8)

  # Scale normalization

  X=[]

  for k, v in data.items():
    if k=='LotShape':
      if v=='Reg':
        X.append(4)
      elif v=='IR1':
        X.append(3)
      elif v=='IR2':
        X.append(2)
      elif v=='IR3':
        x.append(1)
      else:
        x.append(float(v))
  
  # X= [value1, value2, value3, ... ]
  X = np.array(X) # (8, )
  X = X.reshape((1, -1)) # (1, 8)

  # min_max scaling
  scaled_X = x_min_max_scaler.transform(X)
  # print(scaled_X.shape)

  return scaled_X 



@app.route("/")
def home():
  return "<h1>This is your Flask server.</h1>"

@app.route("/predict")
def predict():
  
  return render_template("submit_form.html") # html파일을 직접 띄울 수 있음

@app.route("/result", methods=["POST"])
def result():
  # Read Data
  # Preprocess Data
  # Model Prediction
  # Retrun Prediction
  
  data = request.form # request로 들어온 form 을 읽을 수 있음

  message = ""
  message += "<h1>House Price.</h1>"
  
  for k, v in data.items():
    print(k, v)
    message += k +" : "+v+"</br>"

# data preprocessing
  X=preprocess_data(data) # data : user가 보낸 것

  # pred = model.predict(X) # X:(1, 8)
  pred = reconstructed_model.predict(X) # 미리 훈련시켜 놓은 모델 저장해서 사용한다
  pred = y_min_max_scaler.inverse_transform(pred) # pred: (1,1)

  message+="</br>"
  message+="Predicted price: "+str(pred[0][0]) # message로 보내기 위해서 str 변환

  return message

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


- 필수 파일 X.csv, x.npy, y.npy, html 파일, 학습한 모델 파일만 있으면 플랫폼에 구애받지 않고 배포할 수 있다.